In [55]:
import tensorflow as tf
import pymysql as db
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib

In [56]:
conn = db.connect(host='192.168.2.88', port=3306, user='zach', passwd='1qaz!QAZ', db='old')

In [93]:
cursor = conn.cursor()

In [94]:
query_visits = 'SELECT * from old.matomo_log_visit;'

In [95]:
df_ = pd.read_sql_query(query_visits, conn)

In [96]:
pd.options.display.max_columns = len(df_.columns)
pd.options.display.max_rows = len(df_.columns)

In [97]:
df = df_[['idvisit', 'idsite', 'idvisitor', 'user_id', 'visitor_count_visits',
       'visit_total_actions', 'visitor_days_since_last', 'visit_total_time']]

In [98]:
#replace idvisitor with user_id
df = df.groupby(['user_id', 'idsite']).agg({'user_id': 'count',
                                         'visit_total_actions': 'mean',
                                         'visit_total_time': 'mean',
                                         'visitor_days_since_last': 'mean'}).unstack()

In [99]:
df = df.rename(columns={'user_id' : 'visit_counts'})

In [100]:
#no idsite 1 with user_id
df

visit_counts                          \
idsite                           5      4     6     7    8   
user_id                                                      
!$@@(                          1.0    NaN   NaN   NaN  NaN   
Archit                         2.0    NaN   NaN   NaN  NaN   
Garg                           1.0    NaN   NaN   NaN  NaN   
Isaac                          8.0    NaN   NaN   NaN  NaN   
Mac                           25.0    NaN   NaN   NaN  NaN   
Macbook Pro                    1.0    NaN   NaN   NaN  NaN   
N90 / Aircraft Livery          3.0    NaN   NaN   NaN  NaN   
Sean                           1.0    NaN   NaN   NaN  NaN   
Yishak                         7.0    NaN   NaN   NaN  NaN   
Yishak Mac                     1.0    NaN   NaN   NaN  NaN   
archit                        15.0    2.0   1.0   NaN  NaN   
bdxfv                         66.0    NaN   NaN   NaN  NaN   
chgx                           1.0    NaN   NaN   NaN  NaN   
coolUsername123                NaN    2.0   NaN   NaN  NaN   
dsda                           NaN    NaN   NaN   1.0  NaN   
ergerg                         1.0    NaN   NaN   NaN  NaN   
estn                           NaN    4.0   NaN   NaN  NaN   
fff                            NaN   15.0   NaN   NaN  NaN   
frggfgg                        NaN  127.0   NaN   NaN  NaN   
ggg                            6.0    NaN   NaN   NaN  NaN   
gtest                         16.0    NaN   NaN   NaN  NaN   
hhin                          39.0    NaN   NaN   NaN  NaN   
hyuj                           NaN   14.0   NaN   NaN  NaN   
hyyuuj                         NaN    2.0   NaN   NaN  NaN   
iPad                           3.0    NaN   NaN   NaN  NaN   
iPhone                         9.0    NaN   NaN   NaN  NaN   
ipad                           5.0    NaN   NaN   NaN  NaN   
jjjk                           NaN    1.0   NaN   NaN  NaN   
kien                           NaN    NaN   NaN   7.0  6.0   
mgv                            8.0    NaN   NaN   NaN  NaN   
mhfn                           3.0    NaN   NaN   NaN  NaN   
parthvee                       3.0    3.0   NaN   NaN  NaN   
rty                           42.0    NaN   NaN   NaN  NaN   
sean                          37.0   37.0  12.0  20.0  5.0   
stdw                          20.0    NaN   NaN   NaN  NaN   
tesat                        120.0    NaN   NaN   NaN  NaN   
test                         228.0   31.0   NaN   9.0  9.0   
test1                          3.0    NaN   NaN   NaN  NaN   
testtest                       NaN   10.0   NaN   NaN  NaN   
tets                           2.0    NaN   NaN   NaN  NaN   
trst                           NaN    NaN   NaN   3.0  NaN   
tttt                          18.0    NaN   NaN   NaN  NaN   
txesr                          NaN    NaN   NaN   4.0  NaN   
uitest                        24.0    NaN   NaN   NaN  NaN   
wer                            1.0    NaN   NaN   NaN  NaN   
yhgt                           1.0    NaN   NaN   NaN  NaN   
yishak                         3.0    NaN   NaN   NaN  NaN   
yishak iPad pro                1.0    NaN   NaN   NaN  NaN   
yishak iPhone                  1.0    NaN   NaN   NaN  NaN   
yuii                          99.0    NaN   NaN   NaN  NaN   
yuun                           1.0    NaN   NaN   NaN  NaN   

                      visit_total_actions                                  \
idsite                                  5          4         6          7   
user_id                                                                     
!$@@(                            6.000000        NaN       NaN        NaN   
Archit                           2.500000        NaN       NaN        NaN   
Garg                            14.000000        NaN       NaN        NaN   
Isaac                            5.750000        NaN       NaN        NaN   
Mac                              5.240000        NaN       NaN        NaN   
Macbook Pro                      1.000000        NaN     

In [101]:
#flatten multiindex and assign new column names to df
mi = df.columns
mi.tolist()
ind = pd.Index([e[0] + str(e[1]) for e in mi.tolist()])
df.columns = ind

In [102]:
df.columns

Index(['visit_counts5', 'visit_counts4', 'visit_counts6', 'visit_counts7',
       'visit_counts8', 'visit_total_actions5', 'visit_total_actions4',
       'visit_total_actions6', 'visit_total_actions7', 'visit_total_actions8',
       'visit_total_time5', 'visit_total_time4', 'visit_total_time6',
       'visit_total_time7', 'visit_total_time8', 'visitor_days_since_last5',
       'visitor_days_since_last4', 'visitor_days_since_last6',
       'visitor_days_since_last7', 'visitor_days_since_last8'],
      dtype='object')

In [103]:
len(df.columns)

20

In [104]:
df[['visit_counts5', 'visit_counts4', 'visit_counts6', 'visit_counts7',
       'visit_counts8', 'visit_total_actions5', 'visit_total_actions4',
       'visit_total_actions6', 'visit_total_actions7', 'visit_total_actions8',
       'visit_total_time5', 'visit_total_time4', 'visit_total_time6',
       'visit_total_time7', 'visit_total_time8']] = df[['visit_counts5', 'visit_counts4', 'visit_counts6', 'visit_counts7',
       'visit_counts8', 'visit_total_actions5', 'visit_total_actions4',
       'visit_total_actions6', 'visit_total_actions7', 'visit_total_actions8',
       'visit_total_time5', 'visit_total_time4', 'visit_total_time6',
       'visit_total_time7', 'visit_total_time8']].fillna(value=0)

In [105]:
df[['visitor_days_since_last5',
       'visitor_days_since_last4', 'visitor_days_since_last6',
       'visitor_days_since_last7', 'visitor_days_since_last8']] = df[['visitor_days_since_last5',
       'visitor_days_since_last4', 'visitor_days_since_last6',
       'visitor_days_since_last7', 'visitor_days_since_last8']].fillna(value=999999)

In [106]:
df

,visit_counts5,visit_counts4,visit_counts6,visit_counts7,visit_counts8,visit_total_actions5,visit_total_actions4,visit_total_actions6,visit_total_actions7,visit_total_actions8,visit_total_time5,visit_total_time4,visit_total_time6,visit_total_time7,visit_total_time8,visitor_days_since_last5,visitor_days_since_last4,visitor_days_since_last6,visitor_days_since_last7,visitor_days_since_last8
user_id,,,,,,,,,,,,,,,,,,,,
!$@@(,1.0,0.0,0.0,0.0,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,486.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Archit,2.0,0.0,0.0,0.0,0.0,2.500000,0.000000,0.000000,0.000000,0.000000,108.500000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Garg,1.0,0.0,0.0,0.0,0.0,14.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Isaac,8.0,0.0,0.0,0.0,0.0,5.750000,0.000000,0.000000,0.000000,0.000000,163.750000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Mac,25.0,0.0,0.0,0.0,0.0,5.240000,0.000000,0.000000,0.000000,0.000000,68.480000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Macbook Pro,1.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
N90 / Aircraft Livery,3.0,0.0,0.0,0.0,0.0,3.000000,0.000000,0.000000,0.000000,0.000000,20.666667,0.000000,0.000000,0.000000,0.000000,1.000000,999999.000000,999999.0,999999.000000,999999.000000
Sean,1.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000
Yishak,7.0,0.0,0.0,0.0,0.0,10.142857,0.000000,0.000000,0.000000,0.000000,544.428571,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.0,999999.000000,999999.000000


In [107]:
#need 2 n_neighbors, otherwise the algorithms just finds itself. select second result as closest neighbor
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df)

In [108]:
nbrs

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [109]:
#this id is the get coming in from the application
#user1 = df.loc[b'\x05$v\xa2\xd1&\x16\x0f', :]
user1 = df.iloc[5, :]

In [110]:
user1

visit_counts5                    1.0
visit_counts4                    0.0
visit_counts6                    0.0
visit_counts7                    0.0
visit_counts8                    0.0
visit_total_actions5             1.0
visit_total_actions4             0.0
visit_total_actions6             0.0
visit_total_actions7             0.0
visit_total_actions8             0.0
visit_total_time5                0.0
visit_total_time4                0.0
visit_total_time6                0.0
visit_total_time7                0.0
visit_total_time8                0.0
visitor_days_since_last5         0.0
visitor_days_since_last4    999999.0
visitor_days_since_last6    999999.0
visitor_days_since_last7    999999.0
visitor_days_since_last8    999999.0
Name: Macbook Pro, dtype: float64

In [111]:
# numbers are for each column
distances_1, indices_1 = nbrs.kneighbors([user1])

In [112]:
#indices, first is itself, second is closest
distances_1, indices_1

(array([[ 0.,  0.]]), array([[7, 5]]))

In [124]:
#user_high = df.loc[b'\xa9J\x8f\xe5\xcc\xb1\x9b\xa6', :]
user_high = df.loc['test', :]

In [125]:
user_high

visit_counts5                  228.000000
visit_counts4                   31.000000
visit_counts6                    0.000000
visit_counts7                    9.000000
visit_counts8                    9.000000
visit_total_actions5             8.399123
visit_total_actions4            24.000000
visit_total_actions6             0.000000
visit_total_actions7             1.333333
visit_total_actions8             1.777778
visit_total_time5              115.820175
visit_total_time4              245.612903
visit_total_time6                0.000000
visit_total_time7               84.333333
visit_total_time8               88.000000
visitor_days_since_last5         0.061404
visitor_days_since_last4         0.451613
visitor_days_since_last6    999999.000000
visitor_days_since_last7         0.222222
visitor_days_since_last8         0.777778
Name: test, dtype: float64

In [126]:
distances_2, indices_2 = nbrs.kneighbors([user_high])

In [127]:
#indices, first is itself, second is closest
distances_2, indices_2

(array([[      0.        ,  999999.03830604]]), array([[36, 33]]))

In [128]:
k_neighbor = indices_2[0][1]
k_neighbor

33

In [135]:
df.index.values[k_neighbor]

'sean'

In [134]:
df.iloc[k_neighbor, :]

visit_counts5                37.000000
visit_counts4                37.000000
visit_counts6                12.000000
visit_counts7                20.000000
visit_counts8                 5.000000
visit_total_actions5         21.648649
visit_total_actions4         14.756757
visit_total_actions6          3.916667
visit_total_actions7          5.100000
visit_total_actions8          3.400000
visit_total_time5           231.135135
visit_total_time4            95.945946
visit_total_time6            58.833333
visit_total_time7            65.500000
visit_total_time8            85.400000
visitor_days_since_last5      0.432432
visitor_days_since_last4      0.243243
visitor_days_since_last6      0.000000
visitor_days_since_last7      0.500000
visitor_days_since_last8      1.000000
Name: sean, dtype: float64

In [121]:
df.describe()

,visit_counts5,visit_counts4,visit_counts6,visit_counts7,visit_counts8,visit_total_actions5,visit_total_actions4,visit_total_actions6,visit_total_actions7,visit_total_actions8,visit_total_time5,visit_total_time4,visit_total_time6,visit_total_time7,visit_total_time8,visitor_days_since_last5,visitor_days_since_last4,visitor_days_since_last6,visitor_days_since_last7,visitor_days_since_last8
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,16.196078,4.862745,0.254902,0.862745,0.392157,16.460258,2.208606,0.155229,1.578525,0.467538,135.049082,28.504919,1.173203,16.604575,35.259477,235294.052270,764705.192565,960783.352941,882352.215142,941175.564270
std,38.630568,18.874342,1.683367,3.218817,1.638028,68.816300,6.180845,0.776033,8.840976,2.653363,236.156744,89.645562,8.236704,87.684668,227.671507,428402.804783,428402.763569,196039.015725,325394.928979,237635.031858
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999999.000000,999999.000000,999999.000000,999999.000000
50%,2.000000,0.000000,0.000000,0.000000,0.000000,3.023810,0.000000,0.000000,0.000000,0.000000,68.480000,0.000000,0.000000,0.000000,0.000000,0.076923,999999.000000,999999.000000,999999.000000,999999.000000
75%,12.000000,0.000000,0.000000,0.000000,0.000000,10.133929,0.000000,0.000000,0.000000,0.000000,165.708333,0.000000,0.000000,0.000000,0.000000,2.166667,999999.000000,999999.000000,999999.000000,999999.000000
max,228.000000,127.000000,12.000000,20.000000,9.000000,494.666667,30.333333,4.000000,62.571429,18.666667,1490.333333,493.000000,58.833333,617.000000,1624.833333,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000


In [123]:
#saves nbrs model as knn.pkl
joblib.dump(nbrs, 'knn.pkl')

['knn.pkl']